In [1]:
# Hadley Wickham - coined the term 'split-apply-combine' for describing group operations

In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                   'key2': ['one', 'two', 'one', 'two', 'one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})
df

,key1,key2,data1,data2
0,a,one,1.144600,-1.818531
1,a,two,2.159386,2.019327
2,b,one,2.217549,-0.470690
3,b,two,1.425135,0.057921
4,a,one,0.769435,-0.453212


In [4]:
# compute mean of the data1 column using the labels from key1
# access data1 and call groupby with the column which is a Series at key1

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
# the above grouped variable is now a GroupBy object
# this object has not computed anything yet
# only intermediate data about the group key df['key'] is inferred here
# this object has all the information needed to apply some ops to each of the groups

In [7]:
grouped.mean()

key1
a    1.357807
b    1.821342
Name: data1, dtype: float64

In [8]:
# here the data has been aggregated as per the group key
# results in a new Series indexed by the unique values in key1 column
# result index has name 'key1' as the DF column df['key1'] has the same name

In [9]:
# pass multiple arrays as a list
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     0.957018
      two     2.159386
b     one     2.217549
      two     1.425135
Name: data1, dtype: float64

In [10]:
# result Series has a hierarchical index with unique pair of keys
means.unstack()

key2,one,two
key1,,
a,0.957018,2.159386
b,2.217549,1.425135


In [11]:
# using group keys as arrays of the right length
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [12]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [13]:
df['data1'].groupby([states, years]).mean()

California  2005    2.159386
            2006    2.217549
Ohio        2005    1.284867
            2006    0.769435
Name: data1, dtype: float64

In [14]:
# when grouping information is found in the same DF as the data
# pass column names as the group key
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.357807,-0.084139
b,1.821342,-0.206385


In [15]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.957018 -1.135871
     two   2.159386  2.019327
b    one   2.217549 -0.470690
     two   1.425135  0.057921

In [16]:
# size - returns a Series containing group sizes
# returns the count/size of similar elements
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [17]:
# missing values in a group key will be excluded from the result

### ITERATING OVER GROUPS

In [18]:
# GroupBy supports iteration, generating sequence of 2-tuples
# tuples contain the group name and chunk of data

In [19]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.144600 -1.818531
1    a  two  2.159386  2.019327
4    a  one  0.769435 -0.453212
b
  key1 key2     data1     data2
2    b  one  2.217549 -0.470690
3    b  two  1.425135  0.057921


In [20]:
# in case of multiple keys, 1st element in the tuple will be a tuple of key values
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  1.144600 -1.818531
4    a  one  0.769435 -0.453212
('a', 'two')
  key1 key2     data1     data2
1    a  two  2.159386  2.019327
('b', 'one')
  key1 key2     data1    data2
2    b  one  2.217549 -0.47069
('b', 'two')
  key1 key2     data1     data2
3    b  two  1.425135  0.057921


In [21]:
# computing a dict of data pieces as a one-liner
pieces = dict(list(df.groupby('key1')))

In [22]:
pieces['a']

,key1,key2,data1,data2
0,a,one,1.144600,-1.818531
1,a,two,2.159386,2.019327
4,a,one,0.769435,-0.453212


In [23]:
pieces['b']

,key1,key2,data1,data2
2,b,one,2.217549,-0.470690
3,b,two,1.425135,0.057921


In [24]:
# groupby groups on axis=0
# grouping the columns by dtype
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [25]:
grouped = df.groupby(df.dtypes, axis=1)

In [26]:
for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  1.144600 -1.818531
1  2.159386  2.019327
2  2.217549 -0.470690
3  1.425135  0.057921
4  0.769435 -0.453212
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### SELECTING A COLUMN OR SUBSET OF COLUMNS

In [27]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one  -1.135871
     two   2.019327
b    one  -0.470690
     two   0.057921

In [28]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [29]:
s_grouped.mean()

key1  key2
a     one    -1.135871
      two     2.019327
b     one    -0.470690
      two     0.057921
Name: data2, dtype: float64

### GROUPING WITH DICTS AND SERIES

In [30]:
# grouping information may exist in a form other than array

In [31]:
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people

,a,b,c,d,e
Joe,-1.499115,0.360785,2.136033,-0.544808,-1.200101
Steve,2.561321,-0.027990,0.587808,-1.297354,-0.048079
Wes,-0.882748,-0.909344,0.173169,-0.266423,-0.198867
Jim,-0.712136,0.649146,0.938452,-0.364484,0.615539
Travis,0.216277,1.457909,-0.617405,-0.826201,-0.103501


In [32]:
# adding few NA values
people.iloc[2:3, [1, 2]] = np.nan
people

,a,b,c,d,e
Joe,-1.499115,0.360785,2.136033,-0.544808,-1.200101
Steve,2.561321,-0.027990,0.587808,-1.297354,-0.048079
Wes,-0.882748,NaN,NaN,-0.266423,-0.198867
Jim,-0.712136,0.649146,0.938452,-0.364484,0.615539
Travis,0.216277,1.457909,-0.617405,-0.826201,-0.103501


In [33]:
# using a group correspondence for columns, sum together the columns by group
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

In [34]:
# construct an array from the above dict
by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,1.591224,-2.338431
Steve,-0.709546,2.485252
Wes,-0.266423,-1.081615
Jim,0.573969,0.552550
Travis,-1.443606,1.570685


In [35]:
# same funtionalities hold for Series, viewed as a fixed-size mapping
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [36]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### GROUPING WITH FUNCTIONS

In [37]:
# using Python functions is more generic way of defining a group mapping compared with dict or Series
# any function passed as a group key will be called once per index value
# return values will be used as group names

In [38]:
# group by length of the names
people.groupby(len).sum()

,a,b,c,d,e
3,-3.093999,1.009932,3.074485,-1.175715,-0.783428
5,2.561321,-0.027990,0.587808,-1.297354,-0.048079
6,0.216277,1.457909,-0.617405,-0.826201,-0.103501


In [39]:
# mixing functions with arrays, dicts or Series gets converted to arrays internally
key_list = ['one', 'one', 'one', 'two', 'two']

In [40]:
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.499115  0.360785  2.136033 -0.544808 -1.200101
  two -0.712136  0.649146  0.938452 -0.364484  0.615539
5 one  2.561321 -0.027990  0.587808 -1.297354 -0.048079
6 two  0.216277  1.457909 -0.617405 -0.826201 -0.103501

### GROUPING BY INDEX LEVELS

In [41]:
# convenience for hierarchical indexed dataset is to aggregate using one of the levels of an axis index

In [42]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                     [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])

In [43]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.249825 -2.119359  0.459188  0.084766  1.016968
1      1.172655 -0.462013  1.140063 -0.420844  0.252205
2      0.002540  2.487537  0.410082  1.241047  0.228433
3     -1.863812  0.065999 -1.437945  1.120879  0.822760

In [44]:
# to group by level, pass the level number or name using the level keyword
hier_df.groupby(level="cty", axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## DATA AGGREGATION

In [45]:
# refers to any data transformation that produces scalar values from arrays
# examples - mean, count, min, sum, etc.
# aggregations can be called on any method that is defined on the grouped object
# example - quantiles computes sample quantiles of a Series or a DF's columns

In [46]:
df

,key1,key2,data1,data2
0,a,one,1.144600,-1.818531
1,a,two,2.159386,2.019327
2,b,one,2.217549,-0.470690
3,b,two,1.425135,0.057921
4,a,one,0.769435,-0.453212


In [47]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.956429
b    2.138308
Name: data1, dtype: float64

In [48]:
# to use own aggregation functions, pass any function that aggregates an array to the method
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [49]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.389950,3.837858
b,0.792415,0.528610


In [50]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.357807  0.719085  0.769435  0.957018  1.144600  1.651993   
b      2.0  1.821342  0.560322  1.425135  1.623238  1.821342  2.019446   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     2.159386   3.0 -0.084139  1.945366 -1.818531 -1.135871 -0.453212   
b     2.217549   2.0 -0.206385  0.373784 -0.470690 -0.338537 -0.206385   

                          
           75%       max  
key1                      
a     0.783057  2.019327  
b    -0.074232  0.057921

### COLUMN-WISE AND MULTIPLE FUNCTION APPLICATION

In [51]:
# load tipping dataset and add a tipping percentage column tip_pct
tips = pd.read_csv('../resources/examples/tips.csv')

In [52]:
# add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [53]:
# easy to aggregate a Series or all columns of DF using aggregate with desired function
# aggregating using a different function on specific column, or using multiple functions at once requires more efforts but doable

In [54]:
# grouping tips by day and smoker
grouped = tips.groupby(['day', 'smoker'])

In [55]:
# for descriptive stats, pass the name of the function as a string
grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [56]:
# passing a list of functions will return a DF with column names taken from the functions
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [57]:
# pass a list of (name, function) tuples to be used as the DF column names
# list of 2-tuples as an ordered mapping
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

In [58]:
# DF has more options, as you can specify a list of functions to apply to all of the columns or different functions per column

In [59]:
functions = ['count', 'mean', 'max']

In [60]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [61]:
# the resulting DF has hierarchical columns 
# similar to aggregating each column separately and using concat to glue the results together using column names as key args
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [62]:
# a list of tuples with custom names can be passed

In [63]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [64]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [65]:
# to pass different functions to one or more of the columns
# pass a dict to agg that contains a mapping of column names to any function
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [66]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
             'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

In [67]:
# DF will have hierarchical columns only if multiple functions are applied to at least one column

### RETURNING AGGREGATED DATA WITHOUT ROW INDEXES

In [68]:
# aggregated data comes back with an index which is hierarchical and composed from unique group key combinations
# this can be disabled by passing as_index=False to groupby
# similar can be achieved by reset_index on the result
# using as_index=False method avoids unnecessary computations

In [69]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


### APPLY: GENERAL SPLIT-APPLY-COMBINE

In [70]:
# most general purpose GroupBy method is apply
# apply splits the object being manipulated into pieces, then attempts to concatenate the pieces together

In [71]:
# from tipping dataset select the top five tip_pct values by group
# write a function that selects the rows with the largest values in a particular column
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [72]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [73]:
# group by smoker and call apply with this function
# top function is called on each row group from the DF
# results are glued together using pandas.concat
# pieces are labelled with group names
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [74]:
# functions which take in args can be passed after the function
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [75]:
# note: apply method requires creativity when passing the arg

In [77]:
# describe on a GroupBy object
result = tips.groupby('smoker')['tip_pct'].describe()
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [78]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

In [83]:
# the describe call on GroupBy object is a shortcut for:
# f = lambda x: x.describe()
# grouped.apply(f)

### SUPPRESSING THE GROUP KEYS

In [84]:
# the above resulting object has a hierarchical index 
# formed from the group keys along with indexes of each piece of original object
# disable this by:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


### QUANTILE AND BUCKET ANALYSIS

In [85]:
# cut and qcut - slicing data up into buckets with bins of sample quantiles

In [86]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                      'data2': np.random.randn(1000)})
quartiles = pd.cut(frame.data1, 4)

In [87]:
quartiles[:10]

0     (-0.0426, 1.419]
1    (-1.504, -0.0426]
2     (-0.0426, 1.419]
3     (-0.0426, 1.419]
4    (-1.504, -0.0426]
5     (-0.0426, 1.419]
6     (-2.972, -1.504]
7     (-0.0426, 1.419]
8    (-1.504, -0.0426]
9    (-1.504, -0.0426]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.972, -1.504] < (-1.504, -0.0426] < (-0.0426, 1.419] < (1.419, 2.881]]

In [88]:
# the categorical object returned by cut can be passed to groupby
# compute a set of stats for data2 column:
def get_stats(group):
    return {'min': group.min(),
            'max': group.max(),
            'count': group.count(),
            'mean': group.mean()}

In [90]:
grouped = frame.data2.groupby(quartiles)
grouped

In [91]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.972, -1.504]",61.0,1.474550,-0.061978,-1.850520
"(-1.504, -0.0426]",413.0,3.688749,0.026938,-2.554847
"(-0.0426, 1.419]",441.0,2.756418,-0.014070,-4.241207
"(1.419, 2.881]",85.0,1.657850,-0.278841,-2.548994


In [94]:
# to return quantile numbers
grouping = pd.qcut(frame.data1, 10, labels=False)

In [95]:
grouped = frame.data2.groupby(grouping)
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.869447,0.085452,-2.309325
1,100.0,2.638508,-0.134056,-2.225686
2,100.0,2.967301,0.109306,-2.554847
3,100.0,2.641628,-0.029729,-1.824677
4,100.0,3.688749,0.053640,-2.399020
5,100.0,2.292834,-0.044433,-2.146101
6,100.0,2.491456,-0.207623,-4.241207
7,100.0,2.148708,0.146398,-2.512795
8,100.0,2.756418,-0.030533,-1.967800


### Example: Filling missing values with Group-Specific values

In [96]:
# for cleaning up missing data, usual step would be to use dropna
# to impute (fillna) the null (NA) values using fixed value or some value from data
# fillna serves this purpose

# below is the approach to fill NA values with mean
s = pd.Series(np.random.randn(6))
s[::2] = np.nan
s

0         NaN
1   -0.031955
2         NaN
3   -0.127525
4         NaN
5   -1.896915
dtype: float64

In [98]:
s.fillna(s.mean())

0   -0.685465
1   -0.031955
2   -0.685465
3   -0.127525
4   -0.685465
5   -1.896915
dtype: float64

In [99]:
# fill value to vary by group
# group data and use apply with function that calls fillna on each data chunk

In [100]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon',
          'Nevada', 'California', 'Idaho']

group_key = ['East'] * 4 + ['West'] * 4

In [101]:
data = pd.Series(np.random.randn(8), index=states)
data

Ohio         -0.496738
New York      0.672992
Vermont       0.293054
Florida      -1.969509
Oregon       -1.528434
Nevada       -1.569164
California    0.062855
Idaho        -0.058538
dtype: float64

In [102]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan
data

Ohio         -0.496738
New York      0.672992
Vermont            NaN
Florida      -1.969509
Oregon       -1.528434
Nevada             NaN
California    0.062855
Idaho              NaN
dtype: float64

In [103]:
data.groupby(group_key).mean()

East   -0.597752
West   -0.732789
dtype: float64

In [104]:
# now we can fill the NA values using the group means
fill_mean = lambda x: x.fillna(x.mean())

In [105]:
data.groupby(group_key).apply(fill_mean)

Ohio         -0.496738
New York      0.672992
Vermont      -0.597752
Florida      -1.969509
Oregon       -1.528434
Nevada       -0.732789
California    0.062855
Idaho        -0.732789
dtype: float64

In [108]:
# if the fill values are predefined in the code that vary by group
# use the name attribute of groups which are set internally
fill_values = {'East': 0.5, 'West': -1}

fill_func = lambda x: x.fillna(fill_values[x.name])

In [109]:
data.groupby(group_key).apply(fill_func)

Ohio         -0.496738
New York      0.672992
Vermont       0.500000
Florida      -1.969509
Oregon       -1.528434
Nevada       -1.000000
California    0.062855
Idaho        -1.000000
dtype: float64

### Example: Random Sampling and Permutation

In [110]:
# draw a random sample from a large dataset for Monte Carlo simulation
# number of ways to perform draws, use sample method of Series

In [116]:
# construct a deck of playing cards as an example
# result will be a Series of length 52 whose index contains card names and values
# ace 'A' is kept as 1

In [119]:
# Hearts, Spades, Clubs, Diamonds
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']

cards = []

for suit in suits:
    cards.extend(str(num) + suit for num in base_names)

In [120]:
deck = pd.Series(card_val, index=cards)

In [121]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [122]:
# random drawing a hand of five cards from the deck
def draw(deck, n=5):
    return deck.sample(n)

In [124]:
draw(deck)

9D    9
8S    8
AS    1
8C    8
6H    6
dtype: int64

In [127]:
# draw two random cards from each suit
# now as the suit is the last character of each card name, we can group based on this and use apply
get_suit = lambda card: card[-1] # last letter is suit
deck.groupby(get_suit).apply(draw, n=2)

C  4C      4
   AC      1
D  6D      6
   4D      4
H  AH      1
   2H      2
S  4S      4
   10S    10
dtype: int64

In [129]:
# alternate way to write this
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)

7C     7
4C     4
7D     7
JD    10
KH    10
5H     5
4S     4
KS    10
dtype: int64